# Finaincial Risk Profile RAG v2

Attempting to use markdown loader to chunk based on headings

In [ ]:
pip install langchain langchain_community langchain_core langchain_openai faiss-cpu sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.2/806.2 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [19]:
import pandas as pd
import numpy as np
import json
import re
from tqdm import tqdm
from operator import itemgetter
import pickle
import string


from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.llms import HuggingFaceHub
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_core.output_parsers import JsonOutputParser

In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get("HF_TOKEN")

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Step 0: Load data

In [ ]:
df = pd.read_csv("/content/drive/Shareddrives/ENGS Final Project/Data/Financial Risk Profile Data/Financial Data/CLEAN_merged_frp_data_v2.csv")
df.head()

,Unnamed: 0,company_name,sector,financial_risk_profile,circa_rating,business_description,ffo_to_debt_ltm,debt_to_ebitda_ltm,cfo_to_debt_ltm,focf_to_debt_ltm,dcf_to_debt_ltm,ffo_interest_coverage_ltm,ebitda_to_interest_ltm,ebit_margin_ltm,ebitda_margin_ltm,return_on_capital_ltm
0,0,AAR Corp.,AEROSPACE & DEFENSE,[3] Intermediate,[3] Intermediate risk,AAR Corp. provides products and services to co...,0.18,1.31,0.23,0.11,0.11,2.93,10.30,7.25,8.70,7.05
1,1,Boeing Co.,AEROSPACE & DEFENSE,[4] Significant,[3] Intermediate risk,"The Boeing Company, together with its subsidia...",0.11,10.60,0.16,0.12,0.13,2.42,1.47,1.66,4.05,2.03
2,2,BWX Technologies Inc.,AEROSPACE & DEFENSE,[4] Significant,[3] Intermediate risk,"BWX Technologies, Inc., together with its subs...",0.19,3.44,0.20,0.07,0.14,5.35,8.05,12.20,15.30,8.53
3,3,CACI International Inc.,AEROSPACE & DEFENSE,[3] Intermediate,[3] Intermediate risk,"CACI International Inc, together with its subs...",0.15,2.42,0.17,0.13,0.36,3.20,8.06,8.06,10.00,6.87
4,4,General Dynamics Corp.,AEROSPACE & DEFENSE,[3] Intermediate,[3] Intermediate risk,General Dynamics Corporation operates as an ae...,0.42,1.84,0.51,0.41,0.62,11.80,12.50,8.77,10.80,7.35


### Step 1: Set up RAG vector store

In [ ]:
# Set up markdown splitter
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    ("####", "Header 4"),
    ("#####", "Header 5"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on, strip_headers = False)

# Upload documents
documents = [
    # general crtieria
    '/content/drive/Shareddrives/ENGS Final Project/Data/Financial Risk Profile Data/RAG Knowledge Base/Markdown RAG/general_criteria_markdown_v2.txt',
    # sector-specific guidance
    '/content/drive/Shareddrives/ENGS Final Project/Data/Financial Risk Profile Data/RAG Knowledge Base/Markdown RAG/sector_specific_guidance_markdown_v1.txt',
    # ratios and adjustments
    '/content/drive/Shareddrives/ENGS Final Project/Data/Financial Risk Profile Data/RAG Knowledge Base/Markdown RAG/ratios_and_adjustments.txt',
    # standard volatility table
    '/content/drive/Shareddrives/ENGS Final Project/Data/Financial Risk Profile Data/RAG Knowledge Base/Markdown RAG/Standard Volatility Table.txt',
    # medial volatility table
    '/content/drive/Shareddrives/ENGS Final Project/Data/Financial Risk Profile Data/RAG Knowledge Base/Markdown RAG/Medial Volatility Table.txt',
    # low volatility tale
    '/content/drive/Shareddrives/ENGS Final Project/Data/Financial Risk Profile Data/RAG Knowledge Base/Markdown RAG/Low Volatility Table.txt'
]

# STEP 1: SPLIT ON MARKDOWN HEADERS
md_header_splits = []

for doc in documents:
  doc = open(doc, 'r').read()
  doc_data = markdown_splitter.split_text(doc)
  md_header_splits += doc_data

# STEP 2: ADD RECURSIVE CHARACTER SPLITTER ON PARAGRAPHS/SENTENCES
text_splitter = RecursiveCharacterTextSplitter()

final_doc_splits = text_splitter.split_documents(md_header_splits)
len(final_doc_splits)

195

In [ ]:
final_doc_splits[94]

Document(page_content="### **The Railroad and Package Express Industry Supplemental Ratios**  \nThe Railroad and Package Express Industry Supplemental Ratios: In addition to our analysis of a company's core ratios, we also consider supplemental ratios in order to develop a fuller understanding of a company's credit risk profile and fine tune our cash flow analysis. In our view, a railroad or package express company's inability to meet cash interest payments or a debt maturity would be the most likely cause of a cash default during an industry downturn. Therefore, we consider as supplemental ratios:  \n1. Coverage ratios (FFO + cash interest)/cash interest and EBITDA/interest; and\n2. FOCF to debt (this captures the capital intensity of railroads in particular).", metadata={'Header 2': 'Sector-Specific Guidance', 'Header 3': '**The Railroad and Package Express Industry Supplemental Ratios**'})

## Step 1: Create Vector database and set up retreiver

In [ ]:
vectorstore = FAISS.from_documents(final_doc_splits, embedding=OpenAIEmbeddings())
# vectorstore.save_local("/content/drive/Shareddrives/ENGS Final Project/Data/Financial Risk Profile Data/RAG Knowledge Base/Markdown RAG/vectorstore")

# vectorstore = FAISS.load_local("/content/drive/Shareddrives/ENGS Final Project/Data/Financial Risk Profile Data/RAG Knowledge Base/Markdown RAG/vectorstore", OpenAIEmbeddings(model = 'text-embedding-ada-002'))
retriever = vectorstore.as_retriever()

### Step 1B: Set up multi-prompt structure

In [ ]:
 # retriever = vectorstore.as_retriever()
from langchain_community.llms import HuggingFaceHub

mistral_llm = HuggingFaceHub(
    # repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1",
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 5000,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

mistral_template = """<s>[INST]Use the following context to help you answer the task. If you don't know the answer, please think rationally and answer from your own knowledge base using the methodology provided. You have all the data you need to make this prediction:
{context}

Question: {question} [/INST]
"""
mistral_prompt = ChatPromptTemplate.from_template(mistral_template)

mistral_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | mistral_prompt
    | mistral_llm
    | StrOutputParser()
)
question = "Summarize the sector-specific guidance for the Aerospace and defense industry supplemental ratios to consider."
raw_output = mistral_chain.invoke(question)

mistral_extract_output(raw_output)

NameError: name 'mistral_extract_output' is not defined

In [ ]:
# # Test prompt
# question = "Briefly summarize the sector-specific guidance for the Aerospace and defense industry supplemental ratios to consider."
# chain.invoke(question)

## Step 2: Create prompt template

In [ ]:
def create_prompt1(row):
  return f"""Your goal is to predict the S&P financial risk profile from 1-6 for {row['company_name']}. Take a deep breath and think step by step. Follow the methodology outlined in the knowledge base and the following steps:

  STEP 1: Determine the correct volability table (standard, medial, or low) to use for {row['company_name']} based on the following CICRA and industry:

  CICRA: {row['circa_rating']}
  Industry: {row['sector']}.
  Business description: {row['business_description']}

  The low volatility table will generally apply when a company's CICRA is '1' but can infrequently also apply to a company with a CICRA of '2' if the company exhibits or is expected to exhibit low levels of volatility.
  The medial volatility table will generally apply for a company with a CICRA of '2' but can infrequently also apply to a company with a CICRA of '1' if the company exhibits or is expected to exhibit medial levels of volatility.
  The standard volatility table serves as the relevant benchmark table for all CICRA scores other than '1', but we will always use it for companies with a CICRA of '1' or '2' whose competitive position is assessed as '5' or '6'.
  """

def create_prompt2(row):
  return f"""
  CONTEXT: These are the full volatiltiy tables for each metric:

  Standard Volatility Table:

  Minimal Risk: FFO/Debt greater than or equal to 60%, Debt/EBITDA less than 1.5, FFO/Cash Interest greater than 13, EBITDA/Interest greater than 15, CFO/Debt greater than 50%, FOCF/Debt greater than or equal to 40, DCF/Debt greater than or equal to 25
  Modest Risk: FFO/Debt between 45% and 60%, Debt/EBITDA between 1.5 and 2, FFO/Cash Interest between 9 and 13, EBITDA/Interest between 10 and 15, CFO/Debt between 35% and 50%, FOCF/Debt between 25 and 40, DCF/Debt between 15 and 25
  Intermediate Risk: FFO/Debt between 30% and 45%, Debt/EBITDA between 2 and 3, FFO/Cash Interest between 6 and 9, EBITDA/Interest between 6 and 10, CFO/Debt between 25% and 35%, FOCF/Debt between 15 and 25, DCF/Debt between 10 and 15
  Significant Risk: FFO/Debt between 20% and 30%, Debt/EBITDA between 3 and 4, FFO/Cash Interest between 4 and 6, EBITDA/Interest between 3 and 6, CFO/Debt between 15% and 25%, FOCF/Debt between 10 and 15, DCF/Debt between 5 and 10
  Aggressive Risk: FFO/Debt between 12% and 20%, Debt/EBITDA between 4 and 5, FFO/Cash Interest between 2 and 4, EBITDA/Interest between 2 and 3, CFO/Debt between 10% and 15%, FOCF/Debt between 5 and 10, DCF/Debt between 2 and 5
  Highly Leveraged: FFO/Debt less than 12%, Debt/EBITDA greater than 5, FFO/Cash Interest less than 2, EBITDA/Interest less than 2, CFO/Debt less than 10%, FOCF/Debt less than 5, DCF/Debt less than 2

  Medial Volatility Table:

  Minimal Risk: FFO/Debt greater than or equal to 50%, Debt/EBITDA less than 1.75, FFO/Cash Interest greater than or equal to 10.5, EBITDA/Interest greater than or equal to 14, CFO/Debt greater than or equal to 40%, FOCF/Debt greater thanor equal to 30, DCF/Debt greater than or equal to18
  Modest Risk: FFO/Debt between 35% and 50%, Debt/EBITDA between 1.75 and 2.5, FFO/Cash Interest between 7.5 and 10.5, EBITDA/Interest between 9 and 14, CFO/Debt between 27.5% and 40%, FOCF/Debt between 17.5 and 30, DCF/Debt between 11 and 18
  Intermediate Risk: FFO/Debt between 23% and 35%, Debt/EBITDA between 2.5 and 3.5, FFO/Cash Interest between 5 and 7.5, EBITDA/Interest between 5 and 9, CFO/Debt between 18.5% and 27.5%, FOCF/Debt between 9.5 and 17.5, DCF/Debt between 6.5 and 11
  Significant Risk: FFO/Debt between 13% and 23%, Debt/EBITDA between 3.5 and 4.5, FFO/Cash Interest between 3 and 5, EBITDA/Interest between 2.75 and 5, CFO/Debt between 10.5% and 18.5%, FOCF/Debt between 5 and 9.5, DCF/Debt between 2.5 and 6.5
  Aggressive Risk: FFO/Debt between 9% and 13%, Debt/EBITDA between 4.5 and 5.5, FFO/Cash Interest between 1.75 and 3, EBITDA/Interest between 1.75 and 2.75, CFO/Debt between 7% and 10.5%, FOCF/Debt between 0 and 5, DCF/Debt between -11 and 2.5
  Highly Leveraged: FFO/Debt less than 9%, Debt/EBITDA greater than 5.5, FFO/Cash Interest less than 1.75, CFO/Debt less than 7%, FOCF/Debt less than 0, DCF/Debt less than -11.

  Low Volatility Table:

  Minimal Risk: FFO/Debt greater than or equal to 35%, Debt/EBITDA less than 2, FFO/Cash Interest greater than 8, EBITDA/Interest greater than 13, CFO/Debt greater than 30%, FOCF/Debt greater than or equal to 20, DCF/Debt greater than or equal to 11
  Modest Risk: FFO/Debt between 23% and 35%, Debt/EBITDA between 2 and 3, FFO/Cash Interest between 5 and 8, EBITDA/Interest between 7 and 13, CFO/Debt between 20% and 30%, FOCF/Debt between 10 and 20, DCF/Debt between 7 and 11
  Intermediate Risk: FFO/Debt between 13% and 23%, Debt/EBITDA between 3 and 4, FFO/Cash Interest between 3 and 5, EBITDA/Interest between 4 and 7, CFO/Debt between 12% and 20%, FOCF/Debt between 4 and 10, DCF/Debt between 3 and 7
  Significant Risk: FFO/Debt between 9% and 13%, Debt/EBITDA between 4 and 5, FFO/Cash Interest between 2 and 3, EBITDA/Interest between 2.5 and 4, CFO/Debt between 8% and 12%, FOCF/Debt between 0 and 4, DCF/Debt between 0 and 3
  Aggressive Risk: FFO/Debt between 6% and 9%, Debt/EBITDA between 5 and 6, FFO/Cash Interest between 1.5 and 2, EBITDA/Interest between 1.5 and 2.5, CFO/Debt between 5% and 8%, FOCF/Debt between -10 and 0, DCF/Debt between -20 and 0
  Highly Leveraged: FFO/Debt less than 6%, Debt/EBITDA greater than 6, FFO/Cash Interest less than 1.5, EBITDA/Interest less than 1.5, CFO/Debt less than 5%, FOCF/Debt less than -10, DCF/Debt less than -20.


  STEP 2: Using the corresponding table based on volatility, carefully place ALL of the following core and supplemental key metrics into the correct category. For each core and supplemental metric, display the category it belongs to and the corresponding metric range in the table. Check your answer is logically sound for each.

  Core Financial Metrics:
  FFO to debt: {row['ffo_to_debt_ltm']}
  Debt to EBITDA: {row['debt_to_ebitda_ltm']}

  Supplemental financial metrics:
  FFO cash interest cover: {row['ffo_interest_coverage_ltm']}
  EBITDA to interest: {row['ebitda_to_interest_ltm']}
  CFO to debt: {row['cfo_to_debt_ltm']}
  FOCF to debt: {row['focf_to_debt_ltm']}
  DCF to debt: {row['dcf_to_debt_ltm']}

  Other metrics:
  EBIT margin: {row['ebit_margin_ltm']}
  EBITDA margin: {row['ebitda_margin_ltm']}
  Return on capital: {row['return_on_capital_ltm']}
  CICRA: {row['circa_rating']}
  """

def create_prompt3(row):
  return f"""
  STEP 3: Summarize the sector-specific guidance about which supplemental and core metrics are considered important for the {row['sector'].title()} industry. If the {row['sector'].title()} industry is not mentioned, use the industry and business description to find the most similar sector.
  """

def create_prompt4(row):
  return f"""
  STEP 4: Follow the general corporate methodology and the procedure in the industry-specific guidance to aggregate the key financial ratios and determine the financial risk (1-6). Pay attention to the industry-specific guidance about which metrics are most important for the {row['sector']} industry.
  STEP 4A: First, compare the core ratios (FFO to debt and debt to EBITDA) to the ratio ranges in the relevant benchmark table. If the core ratios result in different cash flow/leverage assessments, select the relevant core ratio based on the industry-specific guidance or whichever the best indicator of a company's future leverage. Next,
  STEP 4B: Next, consider the supplementary ratios. Follow the industry-specific guidance. If the cash flow/leverage assessment(s) indicated by the important supplemental ratio(s) differs from the preliminary cash flow/leverage assessment, we might adjust the preliminary cash flow/leverage assessment by one category in the direction of the cash flow/leverage assessment indicated by the supplemental ratio(s) to derive the adjusted cash flow/leverage assessment. We will make this adjustment if, in our view, the supplemental ratio provides the best indicator of a company's future leverage. If there is more than one important supplemental ratio and they result in different directional deviations from the preliminary cash flow/leverage assessment, we will select one as the relevant supplemental ratio based on which, in our opinion, provides the best indicator of a company's future leverage. We will then make the adjustment outlined above if the selected supplemental ratio differs from the preliminary cash flow/leverage assessment and the selected supplemental ratio provides the best overall indicator of a company's future leverage.
  Your final answer should be a single risk profile from 1 to 6.
  """

def json_final_answer_prompt(row):
  return f"""For your final answer, return a JSON object with 'reasoning' and 'answer' fields.

  Your response should be in the form {{'reasoning': <Methodological explanation as to why you chose that specific rating>, 'answer': <A single category from 1-6> }}"""

### Step 3: Run model on a single company

In [ ]:
def mistral_extract_output(text):
  response = text.split("[/INST]")[-1]
  response = response.split("[\INST]")[-1]
  response = response.split("[\\INST]")[-1]
  response = response.split("[//INST]")[-1]
  # response = response.replace("{", "[")
  # response = response.replace("}", "]")
  return response

In [ ]:
def extract_json_and_answer(input_str):
    # Regular expression to find JSON object within a string
    json_str_match = re.search(r'\{.*\}', input_str, re.DOTALL)

    if json_str_match:
        json_str = json_str_match.group(0)

        try:
            # Parse the JSON string into a Python dictionary
            json_obj = json.loads(json_str)

            # Extract the "answer" field from the JSON object
            answer = json_obj.get('answer', 'No answer found')

            return answer
        except json.JSONDecodeError:
            return -1 # Error parsing JSON
    else:
        return -2 # No JSON object found

In [ ]:
# Prepare df for training run
df['y_pred'] = np.nan
df['reasoning'] = ""

In [ ]:
import time
def wait_until_next_hour():
    """
    Wait until the start of the next hour.
    """
    current_time = time.localtime()
    remaining_minutes = 60 - current_time.tm_min
    sleep_seconds = remaining_minutes * 60 + (60 - current_time.tm_sec)  # Wait until the start of the next hour
    print(f"Waiting for {remaining_minutes} minutes to start the next cycle.")
    time.sleep(sleep_seconds)

In [40]:
# TRAINING LOOP
model = HuggingFaceHub(
    # repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1",
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 5000,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

# Import from most recent checkpoint
# df = pd.read_csv("/content/drive/Shareddrives/ENGS Final Project/Models/Output/RAW_RESULTS_rag_v2_mistral.csv")

# wait_until_next_hour()
# time.sleep(600)

for index, row in tqdm(df.iterrows(), total=df.shape[0], desc=f'Scoring each row in df'):
  if index < 929 or index > 943:
    continue

  # Save progress periodically
  if index % 30 == 0:
    # Save progress
    df.to_csv("/content/drive/Shareddrives/ENGS Final Project/Models/Output/RAW_PARTIAL_RESULTS_rag_v2_mistral.csv")

  # Every 60 observations, wait until the next hour to make sure we're below the API rate limit
  if index > 950 and (index + 1) % 60 == 0:
    wait_until_next_hour()
    time.sleep(600)

  # # STEP 1: DETERMINE THE CORRECT TABLE TO USE
  prompt1_text = create_prompt1(row)
  step1_template = f"<s>[INST] Use the following context to help you determine the financial risk profile for {row['company_name']}:" + """
  {context}

  Take a deep breath and think step by step. Here is the first step:\n {step1} [\INST]
  """
  prompt1 = ChatPromptTemplate.from_template(step1_template)

  chain1 = (
      {"context": retriever, "step1": RunnablePassthrough()}
      | prompt1
      | model
      | StrOutputParser()
  )

  step1_output = chain1.invoke(create_prompt1(row))
  step1_output = mistral_extract_output(step1_output)

  # STEP 2: CATEGORIZE THE FINANCIAL METRICS
  step2_template = f"<s>[INST] Use the following context to help you determine the financial risk profile for {row['company_name']}:" + """
  {context}

  In step 1, we determined the volatility table to use. Here is the reasoning:""" + step1_output + """

  Take a deep breath and think step by step.
  NEXT TASK: {step2} [/INST]"""
  prompt2 = ChatPromptTemplate.from_template(step2_template)
  chain2 = (
      {"context": retriever,
       "step2": RunnablePassthrough()}
      | prompt2
      | model
      | StrOutputParser()
  )
  step2_output = chain2.invoke(create_prompt2(row))
  step2_output = mistral_extract_output(step2_output)

  # STEP 3: SUMMARIZE SECTOR SPECIFIC GUIDANCE
  step3_template = f"""<s>[INST] Briefly summarize the sector-specific guidance about which supplemental and core metrics are considered important for the {row['sector'].title()} industry. Use the following context to help you""" + """
  {context}
  TASK: {step3} [/INST]"""
  prompt3 = ChatPromptTemplate.from_template(step3_template)
  chain3 = (
      {"context": retriever, "step3": RunnablePassthrough()}
      | prompt3
      | model
      | StrOutputParser()
  )

  step3_output = mistral_extract_output(chain3.invoke(create_prompt3(row)))

  # STEP 4: AGGREGATE RATINGS
  step4_template = f"<s>[INST] Use the following context to help you determine the financial risk profile for {row['company_name']}:" + """
  {context}

  Use the output from the first three steps to help you determine the financial risk profile.
  Here is the output from step 1 determining the proper volatility table:""" + step1_output + """
  Here is the output from step 2 assigning each metric to a category on the volatility table:""" + step2_output + """
  Here is the output from step 3 summarizing the industry-specific guidance:""" + step3_output + """

  Take a deep breath and think step by step. YOUR NEXT TASK: {step4} [/INST]"""
  prompt4 = ChatPromptTemplate.from_template(step4_template)
  chain4 = (
      {"context": retriever,
       "step4": RunnablePassthrough()}
      | prompt4
      | model
      | StrOutputParser()
  )

  step4_output = mistral_extract_output(chain4.invoke(create_prompt4(row)))

  # STEP 5: OUTPUT RESULT
  step5_template = """<s>[INST] Use your reasoning in the first four steps to help you determine the financial risk profile:

  Here is the output from step 1 determining the proper volatility table:""" + step1_output + """
  Here is the output from step 2 assigning each metric to a category on the volatility table: """ + step2_output + """
  Here is the output from step 3 summarizing the industry-specific guidance:""" + step3_output + """
  Here is the output from step 4 aggregating the ratings:""" + step4_output + """

  Here is your final task: {step5} [/INST]"""
  prompt5 = ChatPromptTemplate.from_template(step5_template)
  final_chain = (
      {"step5":RunnablePassthrough()}
      | prompt5
      | model
      | StrOutputParser()
  )
  final_output = mistral_extract_output(final_chain.invoke(json_final_answer_prompt(row)))

  final_answer = extract_json_and_answer(final_output)
  df.at[index, 'y_pred'] = final_answer
  df.at[index, 'reasoning'] = "# STEP 1 OUTPUT:\n" + step1_output + "\n # STEP 2 OUTPUT:\n" + step2_output + "\n # STEP 3 OUTPUT:\n" + step3_output + "\n # STEP 4 OUTPUT:\n" + step4_output + "\n # STEP 5 OUTPUT:\n" + final_output

print("finished!")
df.to_csv("/content/drive/Shareddrives/ENGS Final Project/Models/Output/RAW_RESULTS_rag_v2_mistral.csv")

Scoring each row in df: 100%|██████████| 947/947 [00:37<00:00, 24.99it/s]

finished!


In [41]:
df[df[['y_pred', 'financial_risk_profile', 'reasoning']].isna().any(axis = 1)]

,Unnamed: 0.15,Unnamed: 0.14,Unnamed: 0.13,Unnamed: 0.12,Unnamed: 0.11,Unnamed: 0.10,Unnamed: 0.9,Unnamed: 0.8,Unnamed: 0.7,Unnamed: 0.6,...,cfo_to_debt_ltm,focf_to_debt_ltm,dcf_to_debt_ltm,ffo_interest_coverage_ltm,ebitda_to_interest_ltm,ebit_margin_ltm,ebitda_margin_ltm,return_on_capital_ltm,y_pred,reasoning


In [23]:
df[['y_pred', 'financial_risk_profile', 'reasoning']].head(871)

,y_pred,financial_risk_profile,reasoning
0,3.0,[3] Intermediate,# STEP 1 OUTPUT:\n\n Based on the informatio...
1,5.0,[4] Significant,# STEP 1 OUTPUT:\n\n Based on the provided i...
2,5.0,[4] Significant,# STEP 1 OUTPUT:\n\n Based on the provided i...
3,4.0,[3] Intermediate,# STEP 1 OUTPUT:\n\n Based on the provided i...
4,3.0,[3] Intermediate,# STEP 1 OUTPUT:\n\n Based on the provided i...
...,...,...,...
866,4,[1] Minimal,# STEP 1 OUTPUT:\n\n Based on the informatio...
867,5,[1] Minimal,# STEP 1 OUTPUT:\n\n Based on the provided i...
868,5,[6] Highly leveraged,# STEP 1 OUTPUT:\n\n Based on the provided i...
869,3,[3] Intermediate,# STEP 1 OUTPUT:\n\n Based on the provided i...


In [ ]:
from IPython.display import Markdown, display
display(Markdown(df['reasoning'].iloc[14]))

In [ ]:
df.to_csv("/content/drive/Shareddrives/ENGS Final Project/Models/Output/RAW_PARTIAL_RESULTS_rag_v2_mistral.csv")

NameError: name 'df' is not defined